In [11]:
import os, csv, json, requests
import glob
import pycountry
import pandas as pd
import numpy as np
from datetime import date

In [6]:
#-----------------PVGIS create URLs----------------------------------                   
url = "https://re.jrc.ec.europa.eu/api/PVcalc?lat=45&lon=8&peakpower=1&loss=14&outputformat=json"

response = requests.get(url)

In [7]:
jsondata = response.json()
jsondata

{'inputs': {'location': {'latitude': 45.0,
   'longitude': 8.0,
   'elevation': 250.0},
  'meteo_data': {'radiation_db': 'PVGIS-SARAH',
   'meteo_db': 'ERA-Interim',
   'year_min': 2005,
   'year_max': 2016,
   'use_horizon': True,
   'horizon_db': 'DEM-calculated'},
  'mounting_system': {'fixed': {'slope': {'value': 0, 'optimal': False},
    'azimuth': {'value': 0, 'optimal': False},
    'type': 'free-standing'}},
  'pv_module': {'technology': 'c-Si', 'peak_power': 1.0, 'system_loss': 14.0},
  'economic_data': {'system_cost': None, 'interest': None, 'lifetime': None}},
 'outputs': {'monthly': {'fixed': [{'month': 1,
     'E_d': 1.21,
     'E_m': 37.38,
     'H(i)_d': 1.54,
     'H(i)_m': 47.75,
     'SD_m': 4.31},
    {'month': 2,
     'E_d': 1.88,
     'E_m': 52.57,
     'H(i)_d': 2.33,
     'H(i)_m': 65.3,
     'SD_m': 6.74},
    {'month': 3,
     'E_d': 3.06,
     'E_m': 94.91,
     'H(i)_d': 3.79,
     'H(i)_m': 117.42,
     'SD_m': 10.93},
    {'month': 4,
     'E_d': 3.84,
     

In [8]:
# Extraire la valeur de E_y
E_y_value = jsondata["outputs"]["totals"]["fixed"]["E_y"]
E_y_value

1137.32

In [9]:
# Nominal power of the PV system, in kW. 1kWp / m²

# PVgis technology. Choices are: "crystSi", "CIS", "CdTe" and "Unknown"
# Ecoinvent technology : "single-Si", "multi-Si", "a-Si", "ribbon-Si", "CIS", "CdTe"

# Type of mounting of the PV modules. Choices are: "free" for free-standing and "building" for building-integrated.
# Ecoinvent mounting : "facade"-"flat-roof"-"slanted-roof" = building, "open ground" = free

# Inclination angle from horizontal plane of the (fixed) PV system.
# Ecoinvent inclination : "facade" = 90°, "-roof" = 0°, "open ground" = no value

# Calculate the optimum inclination AND orientation angles. Value of 1 for "yes". All other values (or no value) mean "no".
# Ecoinvent : "open ground" = 1, no value for others

In [12]:
import pickle

with open(r"pv_detailsSSP2-RCP19-2050.pickle", "rb") as input_file:
        pv_pickle = pickle.load(input_file)

In [13]:
pv_pickle

,Market Dataset,ecoinvent location,Market group location,Peak power (kWp),Installation,Technology,Placement,Build,Placement2
0,"market for electricity, low voltage, renewable...",CH,NaN,3,slanted-roof,single-Si,panel,mounted,on roof
1,"market for electricity, low voltage, renewable...",CH,NaN,3,slanted-roof,multi-Si,panel,mounted,on roof
0,"market for electricity, low voltage",BJ,WAF,3,slanted-roof,CIS,laminated,integrated,on roof
1,"market for electricity, low voltage",BJ,WAF,3,slanted-roof,CdTe,panel,mounted,on roof
2,"market for electricity, low voltage",BJ,WAF,3,slanted-roof,micro-Si,laminated,integrated,on roof
...,...,...,...,...,...,...,...,...,...
14116,"market for electricity, high voltage",CN-CSG,CHN,570,open ground,single-Si,on open ground,None,None
14117,"market for electricity, high voltage",CN-CSG,CHN,93,slanted-roof,multi-Si,laminated,integrated,on roof
14118,"market for electricity, high voltage",CN-CSG,CHN,93,slanted-roof,multi-Si,panel,mounted,on roof
14119,"market for electricity, high voltage",CN-CSG,CHN,93,slanted-roof,single-Si,laminated,integrated,on roof


In [14]:
str(pv_pickle.iloc[0]["Peak power (kWp)"])

'3'

In [15]:
iRow = 0

# Test de la valeur dans la colonne de la ligne spécifiée
valeur_testee = pv_pickle.iloc[iRow]["Technology"]

# Assignation de la variable résultat en utilisant une expression conditionnelle ternaire
variable_resultat = ("La valeur contient 'Si'" if "Si" in valeur_testee
                    else "La valeur est égale à 'CIS'" if valeur_testee == "CIS"
                    else "La valeur est égale à 'CdTe'" if valeur_testee == "CdTe"
                    else "Aucune condition n'est remplie")

# Afficher la variable résultat
print(variable_resultat)

La valeur contient 'Si'


In [44]:
iRow = 0

# Nominal power of the PV system, in kW. 1kWp / m²
# peakpower
peakpower = pv_pickle.iloc[iRow]["Peak power (kWp)"]

# PVgis technology. Choices are: "crystSi", "CIS", "CdTe" and "Unknown"
# pvtechchoice
# Ecoinvent technology : "single-Si", "multi-Si", "a-Si", "ribbon-Si", "CIS", "CdTe"

Tech_pickle = pv_pickle.iloc[iRow]["Technology"]
pvtechchoice = ("crystSi" if "Si" in Tech_pickle
                    else "CIS" if valeur_testee == "CIS"
                    else "CdTe" if valeur_testee == "CdTe"
                    else "Unknown")

# Type of mounting of the PV modules are: "free" for free-standing and "building" for building-integrated.
# mountingplace
# Ecoinvent mounting : "facade"-"flat-roof"-"slanted-roof" = building, "open ground" = free

Mounting_pickle = pv_pickle.iloc[iRow]["Installation"]
mountingplace = ("building" if "roof" or "facade" in Mounting_pickle
                    else "free")

# Inclination angle from horizontal plane of the (fixed) PV system.
# angle
# Ecoinvent inclination : "facade" = 90°, "-roof" = 0°, "open ground" = no value

angle = ("0" if "roof" in Mounting_pickle
                    else "90" if "facade" in Mounting_pickle
                    else "")

# Calculate the optimum inclination AND orientation angles. Value of 1 for "yes". All other values (or no value) mean "no".
# optimalinclination
# Ecoinvent : "open ground" = 1, no value for others

optimalinclination = ("1" if "open ground" in Mounting_pickle
                    else "")
p1 = "&peakpower="
p2 = "&pvtechchoice="
p3 = "&mountingplace="
p4 = "&angle="
p5 = "&optimalinclination="
mp = "&loss=14&outputformat=json"
base_url = "https://re.jrc.ec.europa.eu/api/PVcalc?lat=45.32&lon=8.43"

url_test = base_url+p1+str(peakpower)+p2+pvtechchoice+p3+mountingplace+p4+angle+p5+optimalinclination+mp
url_test

'https://re.jrc.ec.europa.eu/api/PVcalc?lat=45.32&lon=8.43&peakpower=3&pvtechchoice=crystSi&mountingplace=building&angle=0&optimalinclination=&loss=14&outputformat=json'

In [45]:
response_test = requests.get(url_test)
jsondata_test = response_test.json()
E_y_value = jsondata_test["outputs"]["totals"]["fixed"]["E_y"]
E_y_value
kwh_per_sqmeter = E_y_value/peakpower
kwh_per_sqmeter

1087.3700000000001

In [10]:
pv_pickle.to_csv("pv_pickle.csv")

In [55]:
base_url_coord = "https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?"

In [60]:
for index, row in df_coord.iterrows():
    # Extraire les valeurs de latitude et de longitude de la ligne actuelle
    lat = "lat="+"{:.2f}".format(row['Latitude'])+"&"
    lon = "lon="+"{:.2f}".format(row['Longitude'])
    url_test = base_url_coord+lat+lon+p1+str(peakpower)+p2+pvtechchoice+p3+mountingplace+p4+angle+p5+optimalinclination+mp
    print(url_test)
    response_test = requests.get(url_test)
    jsondata_test = response_test.json()
    print(response_test.json())
#   jsondata_test
    #E_y_value = jsondata_test["outputs"]["totals"]["fixed"]["E_y"]
    #E_y_value
  #  kwh_per_sqmeter = E_y_value/peakpower
  #  kwh_per_sqmeter
    

https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?lat=70.62&lon=-100.34&peakpower=3&pvtechchoice=crystSi&mountingplace=building&angle=0&optimalinclination=&loss=14&outputformat=json
{'inputs': {'location': {'latitude': 70.62, 'longitude': -100.34, 'elevation': 6.0}, 'meteo_data': {'radiation_db': 'PVGIS-ERA5', 'meteo_db': 'ERA5', 'year_min': 2005, 'year_max': 2020, 'use_horizon': True, 'horizon_db': 'DEM-calculated'}, 'mounting_system': {'fixed': {'slope': {'value': 0, 'optimal': False}, 'azimuth': {'value': 0, 'optimal': False}, 'type': 'building-integrated'}}, 'pv_module': {'technology': 'c-Si', 'peak_power': 3.0, 'system_loss': 14.0}, 'economic_data': {'system_cost': None, 'interest': None, 'lifetime': None}}, 'outputs': {'monthly': {'fixed': [{'month': 1, 'E_d': 0.0, 'E_m': 0.09, 'H(i)_d': 0.0, 'H(i)_m': 0.15, 'SD_m': 0.02}, {'month': 2, 'E_d': 0.71, 'E_m': 19.94, 'H(i)_d': 0.38, 'H(i)_m': 10.54, 'SD_m': 0.78}, {'month': 3, 'E_d': 4.65, 'E_m': 144.3, 'H(i)_d': 1.87, 'H(i)_m': 57.9, 'SD_m

In [54]:
from urllib.parse import quote

# Exemple de coordonnées
latitude = -34.397
longitude = 150.644

# Encodage des coordonnées pour les inclure dans l'URL de la requête GET
latitude_encoded = quote(str(latitude))
longitude_encoded = quote(str(longitude))

# Construire l'URL de la requête GET avec les coordonnées encodées
url = f"https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?lat={latitude_encoded}&lon={longitude_encoded}&peakpower=1&loss=14&outputformat=json"
print(url)
# Effectuer la requête GET avec cette URL
# Utilisez la bibliothèque appropriée pour effectuer des requêtes HTTP, comme requests pour Python
response = requests.get(url)
response.json()

https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?lat=-34.397&lon=150.644&peakpower=1&loss=14&outputformat=json


{'inputs': {'location': {'latitude': -34.397,
   'longitude': 150.644,
   'elevation': 387.0},
  'meteo_data': {'radiation_db': 'PVGIS-ERA5',
   'meteo_db': 'ERA5',
   'year_min': 2005,
   'year_max': 2020,
   'use_horizon': True,
   'horizon_db': 'DEM-calculated'},
  'mounting_system': {'fixed': {'slope': {'value': 0, 'optimal': False},
    'azimuth': {'value': 0, 'optimal': False},
    'type': 'free-standing'}},
  'pv_module': {'technology': 'c-Si', 'peak_power': 1.0, 'system_loss': 14.0},
  'economic_data': {'system_cost': None, 'interest': None, 'lifetime': None}},
 'outputs': {'monthly': {'fixed': [{'month': 1,
     'E_d': 4.76,
     'E_m': 147.52,
     'H(i)_d': 6.29,
     'H(i)_m': 194.85,
     'SD_m': 11.1},
    {'month': 2,
     'E_d': 4.13,
     'E_m': 115.7,
     'H(i)_d': 5.41,
     'H(i)_m': 151.53,
     'SD_m': 9.2},
    {'month': 3,
     'E_d': 3.44,
     'E_m': 106.78,
     'H(i)_d': 4.5,
     'H(i)_m': 139.35,
     'SD_m': 10.31},
    {'month': 4,
     'E_d': 2.85,
   